# Moment Applied to Price data alone (monthly)

In [32]:
# Constants
#!pip install numpy pandas matplotlib tqdm
#!pip install git+https://github.com/moment-timeseries-foundation-model/moment.git
FORECAST_HORIZON = 12
SEQ_LEN = 256
BATCH_SIZE = 10
MAX_EPOCH = 2
DATASET_PATH = './zone_data_1A_PRICE_MONTHLY.csv'



In [33]:
from momentfm import MOMENTPipeline
#Initialise the model.
model = MOMENTPipeline.from_pretrained(
    "AutonLab/MOMENT-1-large", 
    model_kwargs={
        #can load in (1) reconstruction, (2) embedding, (3) forecasting, and (4) classification tasks
        'task_name': 'forecasting',

        # Number of steps to forecast (original is 192)
        'forecast_horizon': FORECAST_HORIZON,

        #???
        'head_dropout': 0.1,

        #???
        'weight_decay': 0,
        'freeze_encoder': True, # Freeze the patch embedding layer
        'freeze_embedder': True, # Freeze the transformer encoder
        'freeze_head': False, # The linear forecasting head must be trained
    },
)


model.init()
print("Patch Length:", model.patch_len)
# Parameters that need to be trained
print("Unfrozen parameters:")
for name, param in model.named_parameters():    
    if param.requires_grad:
        print('    ', name)

Patch Length: 8
Unfrozen parameters:
     head.linear.weight
     head.linear.bias


In [34]:
# Do a "forward pass through moment", and then examine output. MOMENT model needs 3 inputs:
# 1. An input timeseries of 512(change later??) timesteps
# 2. Two "optional" masks, both of length 512
# "The input mask is utilized to regulate the time steps or patches that the model should attend to. For instance, in the case of shorter time series, you may opt not to attend to padding. To implement this, you can provide an input mask with zeros in the padded locations."
#"The second mask, referred to simply as mask, denotes masked or unobserved values. We employ mask tokens to replace all patches containing any masked time step (for further details, refer to Section 3.2 in our paper). MOMENT can attend to these mask tokens during reconstruction."

from pprint import pprint
import torch

# takes in tensor of shape [batchsize, n_channels, context_length]
x = torch.randn(10, 1, SEQ_LEN)
output = model(x)

#model return "TimeseriesOutputs" Object
pprint(output)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x32768 and 65536x12)

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import torch
FORECAST_HORIZON = 192
SEQ_LEN = 512

class ETThDataset(Dataset):
    def __init__(self, data, forecast_horizon=FORECAST_HORIZON, seq_len=SEQ_LEN):
        self.data = data
        self.forecast_horizon = forecast_horizon
        self.seq_len = seq_len
        
    def __len__(self):
       return max(0, len(self.data) - self.seq_len - self.forecast_horizon)
    
    def __getitem__(self, idx):
        # Ensure the index is within the bounds of the dataset
        if idx + self.seq_len + self.forecast_horizon > len(self.data):
            raise IndexError("Index out of bounds for dataset length.")
        
        start_idx = idx
        end_idx = start_idx + self.seq_len
        target_end_idx = end_idx + self.forecast_horizon
        
        # Extract the input and target sequences
        x = torch.tensor(self.data.iloc[start_idx:end_idx, :].values, dtype=torch.float32)
        y = torch.tensor(self.data.iloc[end_idx:target_end_idx, -1].values, dtype=torch.float32)
        y = y.unsqueeze(0)
        
        # Assuming all data points are valid
        input_mask = np.ones(self.seq_len, dtype=np.float32)
        
        return x, y, input_mask

In [ ]:
from typing import Optional

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


class MyInformerDataset:
    def __init__(
        self,
        forecast_horizon: Optional[int] = FORECAST_HORIZON,
        data_split: str = "train",
        data_stride_len: int = 1,
        task_name: str = "forecasting",
        random_seed: int = 42,
    ):
        """
        Parameters
        ----------
        forecast_horizon : int
            Length of the prediction sequence.
        data_split : str
            Split of the dataset, 'train' or 'test'.
        data_stride_len : int
            Stride length when generating consecutive
            time series windows.
        task_name : str
            The task that the dataset is used for. One of
            'forecasting', or  'imputation'.
        random_seed : int
            Random seed for reproducibility.
        """

        self.seq_len = SEQ_LEN
        self.forecast_horizon = forecast_horizon
        self.full_file_path_and_name = DATASET_PATH
        self.data_split = data_split
        self.data_stride_len = data_stride_len
        self.task_name = task_name
        self.random_seed = random_seed

        # Read data
        self._read_data()

    def _get_borders(self):
        n_train = 12 * 30 * 24
        n_val = 4 * 30 * 24
        n_test = 4 * 30 * 24

        train_end = n_train
        val_end = n_train + n_val
        test_start = val_end - self.seq_len
        test_end = test_start + n_test + self.seq_len

        train = slice(0, train_end)
        test = slice(test_start, test_end)

        return train, test

    def _read_data(self):
        self.scaler = StandardScaler()
        df = pd.read_csv(self.full_file_path_and_name)
        self.length_timeseries_original = df.shape[0]
        self.n_channels = df.shape[1] - 1

        df.drop(columns=["date"], inplace=True)
        #modify to remove copy
        df = df.infer_objects().interpolate(method="cubic")

        data_splits = self._get_borders()

        train_data = df[data_splits[0]]
        self.scaler.fit(train_data.values)
        df = self.scaler.transform(df.values)

        if self.data_split == "train":
            self.data = df[data_splits[0], :]
        elif self.data_split == "test":
            self.data = df[data_splits[1], :]

        self.length_timeseries = self.data.shape[0]

    def __getitem__(self, index):
        seq_start = self.data_stride_len * index
        seq_end = seq_start + self.seq_len
        input_mask = np.ones(self.seq_len)

        if self.task_name == "forecasting":
            pred_end = seq_end + self.forecast_horizon

            if pred_end > self.length_timeseries:
                pred_end = self.length_timeseries
                seq_end = seq_end - self.forecast_horizon
                seq_start = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T
            forecast = self.data[seq_end:pred_end, :].T

            return timeseries, forecast, input_mask

        elif self.task_name == "imputation":
            if seq_end > self.length_timeseries:
                seq_end = self.length_timeseries
                seq_end = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T

            return timeseries, input_mask

    def __len__(self):
        if self.task_name == "imputation":
            return (self.length_timeseries - self.seq_len) // self.data_stride_len + 1
        elif self.task_name == "forecasting":
            return (self.length_timeseries - self.seq_len - self.forecast_horizon) // self.data_stride_len + 1

## Model FineTuning
"The forecasting head is randomly initialized, so it must be trained on your data."


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.cuda.amp
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR
from tqdm import tqdm

from momentfm.utils.utils import control_randomness
#from momentfm.data.informer_dataset import InformerDataset
from momentfm.utils.forecasting_metrics import get_forecasting_metrics

# Set random seeds for PyTorch, Numpy etc.
control_randomness(seed=13) 

# Load data
# Specify the path to your CSV file
#csv_file_path = '../data/ETTh1.csv'
df = pd.read_csv(DATASET_PATH)
#df = df.drop(columns=["date"])
print(df)
print(len(df))

train_size = int(0.8 * len(df))
print(train_size)
df_train = df.iloc[:train_size]
df_test = df.iloc[train_size:-(SEQ_LEN+FORECAST_HORIZON)]

train_dataset = MyInformerDataset(data_split="train", random_seed=13, forecast_horizon=FORECAST_HORIZON)
test_dataset = MyInformerDataset(data_split="test", random_seed=13, forecast_horizon=FORECAST_HORIZON)
#train_dataset = ETThDataset(df_train, forecast_horizon=FORECAST_HORIZON)
#test_dataset = ETThDataset(df_test, forecast_horizon=FORECAST_HORIZON)


# doesn't Affects running time much! (64 default) but less makes it speed up epochs to match data loading
batch_size = BATCH_SIZE
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cur_epoch = 0
max_epoch = MAX_EPOCH

# Move the model to the GPU
model = model.to(device)

# Move the loss function to the GPU
criterion = criterion.to(device)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Create a OneCycleLR scheduler
max_lr = 1e-4
total_steps = len(train_loader) * max_epoch
scheduler = OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps, pct_start=0.3)

# Gradient clipping value
max_norm = 5.0

while cur_epoch < max_epoch:
    losses = []
    for timeseries, forecast, input_mask in tqdm(train_loader, total=len(train_loader)):
        # Move the data to the GPU
        timeseries = timeseries.float().to(device)
        input_mask = input_mask.to(device)
        forecast = forecast.float().to(device)

        with torch.cuda.amp.autocast():
            print(timeseries.shape)
            output = model(timeseries, input_mask)
        
        loss = criterion(output.forecast, forecast)

        # Scales the loss for mixed precision training
        scaler.scale(loss).backward()

        # Clip gradients
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        losses.append(loss.item())

    losses = np.array(losses)
    average_loss = np.average(losses)
    print(f"Epoch {cur_epoch}: Train loss: {average_loss:.3f}")

    # Step the learning rate scheduler
    scheduler.step()
    cur_epoch += 1
    
    # Evaluate the model on the test split
    trues, preds, histories, losses = [], [], [], []
    model.eval()
    with torch.no_grad():
        for timeseries, forecast, input_mask in tqdm(test_loader):
        # Move the data to the GPU
            timeseries = timeseries.float().to(device)
            input_mask = input_mask.to(device)
            forecast = forecast.float().to(device)

            with torch.cuda.amp.autocast():
                print(timeseries.shape)
                output = model(timeseries, input_mask)
            
            loss = criterion(output.forecast, forecast)                
            losses.append(loss.item())

            trues.append(forecast.detach().cpu().numpy())
            preds.append(output.forecast.detach().cpu().numpy())
            histories.append(timeseries.detach().cpu().numpy())
    
    losses = np.array(losses)
    average_loss = np.average(losses)
    model.train()

    trues = np.concatenate(trues, axis=0)
    preds = np.concatenate(preds, axis=0)
    histories = np.concatenate(histories, axis=0)
    
    metrics = get_forecasting_metrics(y=trues, y_hat=preds, reduction='mean')

    print(f"Epoch {cur_epoch}: Test MSE: {metrics.mse:.3f} | Test MAE: {metrics.mae:.3f}")

     Unnamed: 0        date         mean         std
0             0  2009-04-30  2150.000000  191.485422
1             1  2009-05-31  2073.684211  299.707460
2             2  2009-06-30  2106.350000  267.471690
3             3  2009-07-31  2306.540897  237.066784
4             4  2009-08-31  2140.525750  259.103054
..          ...         ...          ...         ...
176         176  2023-12-31  4006.749500  214.042895
177         177  2024-01-31  4040.147273  227.444588
178         178  2024-02-29  4002.058824  205.437883
179         179  2024-03-31  4013.611111  202.947440
180         180  2024-04-30  3904.163333   78.133220

[181 rows x 4 columns]
181
144


ValueError: __len__() should return >= 0

In [ ]:
# visualise it!

import matplotlib.pyplot as plt

# Assuming histories, trues, and preds are your lists containing the data
# Extracting the first data point

channel_idx = np.random.randint(0, trues.shape[1]) # There are 7 channels in this dataset
time_index = np.random.randint(0, trues.shape[0]) 

history = histories[time_index, channel_idx, :] 
true = trues[time_index, channel_idx, :]
pred = preds[time_index, channel_idx, :]

plt.figure(figsize=(12, 4))

# Plotting the first time series from history
plt.plot(range(len(history)), history, label='History (512 timesteps)', c='darkblue')

# Plotting ground truth and prediction
num_forecasts = len(true)

offset = len(history)
plt.plot(range(offset, offset + len(true)), true, label='Ground Truth (192 timesteps)', color='darkblue', linestyle='--', alpha=0.5)
plt.plot(range(offset, offset + len(pred)), pred, label='Forecast (192 timesteps)', color='red', linestyle='--')

plt.title(f"ETTh1 (Hourly) -- (idx={time_index}, channel={channel_idx})", fontsize=18)
plt.xlabel('Time', fontsize=14)
plt.ylabel('Value', fontsize=14)
plt.legend(fontsize=14)
plt.show()